In [35]:
import torch
from torch import nn
from torchvision import datasets, transforms

## 1. Parameters

In [36]:
# dataset
input_shape = 300
num_classes = 2

# hyper
batch_size = 4
num_epochs = 5
learning_rate = 0.001

# gpu
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. Dataset and Dataloader

In [37]:
import splitfolders
raw = './raw'
split = './processed'
splitfolders.ratio(raw, split, seed = 1337, ratio = (0.7, 0.3))

Copying files: 258 files [00:00, 413.78 files/s]


In [38]:
data_transfrom = transforms.Compose([  
    transforms.ToTensor(),             
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((300, 300)),
])
 
trainset = datasets.ImageFolder('./processed/train', transform = data_transfrom) 
testset = datasets.ImageFolder('./processed/val', transform = data_transfrom) 

classes = ('AMD', 'NORMAL')

In [39]:
trainloader = torch.utils.data.DataLoader(dataset = trainset, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          num_workers = 1) 
testloader = torch.utils.data.DataLoader(dataset = testset, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         num_workers = 1)

In [40]:
images, labels = next(iter(trainloader))

In [41]:
images.shape

torch.Size([4, 3, 300, 300])

## 3. Model Arch

In [42]:
class CNN(nn.Module):
    def __init__(self, input_shape, in_channels, num_classes):
        super(CNN, self).__init__()
        # conv2d: (b, 3, 300, 300) => (b, 9, 300, 300)
        # maxpool2d: (b, 9, 300, 300) => (b, 9, 150, 150)
        self.cnn1 = nn.Sequential(nn.Conv2d(in_channels = in_channels, out_channels = 9, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(9),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        # conv2d: (b, 9, 150, 150) => (b, 27, 150, 150)
        # maxpool2d: (b, 27, 150, 150) => (b, 27, 75, 75)
        self.cnn2 = nn.Sequential(nn.Conv2d(in_channels = 9, out_channels = 27, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(27),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        # (b, 27, 75, 75) => (b, 27*75*75)
        # (b, 27*75*75) => (b, 2)
        self.fc = nn.Linear(27*(input_shape//4)*(input_shape//4), num_classes)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.cnn2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

### Torchsummary

In [43]:
from torchsummary import summary

In [44]:
model = CNN(input_shape = 300, in_channels = 3, num_classes = 2)

In [45]:
summary(model, input_size = (3, 300, 300), batch_size = batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [4, 9, 300, 300]             684
       BatchNorm2d-2           [4, 9, 300, 300]              18
              ReLU-3           [4, 9, 300, 300]               0
         MaxPool2d-4           [4, 9, 150, 150]               0
            Conv2d-5          [4, 27, 150, 150]           6,102
       BatchNorm2d-6          [4, 27, 150, 150]              54
              ReLU-7          [4, 27, 150, 150]               0
         MaxPool2d-8            [4, 27, 75, 75]               0
            Linear-9                     [4, 2]         303,752
Total params: 310,610
Trainable params: 310,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.12
Forward/backward pass size (MB): 140.59
Params size (MB): 1.18
Estimated Total Size (MB): 145.90
---------------------------------------

## 4. Model Train

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [47]:
total_batch = len(trainloader)

In [48]:
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(trainloader):
#         images = images.to(device)
#         labels = labels.to(device)
        
        out = model(images)
        loss = criterion(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 10 == 0:
            print(f'{epoch+1}/{num_epochs}, {batch_idx+1}/{total_batch}: {loss.item():.4f}')

print('Finished Training')

1/5, 10/45: 0.0000
1/5, 20/45: 0.9935
1/5, 30/45: 0.5984
1/5, 40/45: 0.4045
2/5, 10/45: 0.0365
2/5, 20/45: 0.8584
2/5, 30/45: 2.1471
2/5, 40/45: 0.0002
3/5, 10/45: 2.9572
3/5, 20/45: 0.0016
3/5, 30/45: 0.0052
3/5, 40/45: 0.0069
4/5, 10/45: 0.2080
4/5, 20/45: 0.0007
4/5, 30/45: 0.0242
4/5, 40/45: 0.0025
5/5, 10/45: 0.2814
5/5, 20/45: 0.0777
5/5, 30/45: 4.3173
5/5, 40/45: 0.0003
Finished Training


## 5. Model Evaluation

In [49]:
total = 0
correct = 0
for images, labels in testloader:
#    images = images.to(device)
#    labels = labels.to(device)
    out = model(images)
    preds = torch.argmax(out, dim = 1)
    
    total += images.size(0)
    correct += (preds == labels).sum().item()
print(f'{correct}/{total} = {correct/total}')

71/79 = 0.8987341772151899


In [50]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for images, labels in testloader:
        out = model(images)
        _, predictions = torch.max(out, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: AMD   is 64.7 %
Accuracy for class: NORMAL is 96.8 %


## 6. Model Save

In [51]:
#torch.save(model.state_dict(),'cnn_1.ckpt')